In [17]:
import os
import sys
sys.path.append(os.path.realpath('../..'))
import aoc
aoc.set_date(2016,9)

AttributeError: module 'aoc' has no attribute 'set_date'

#Part 1
- ADVENT contains no markers and decompresses to itself with no changes, resulting in a decompressed length of 6.
- A(1x5)BC repeats only the B a total of 5 times, becoming ABBBBBC for a decompressed length of 7.
- (3x3)XYZ becomes XYZXYZXYZ for a decompressed length of 9.
- A(2x2)BCD(2x2)EFG doubles the BC and EF, becoming ABCBCDEFEFG for a decompressed length of 11.
- (6x1)(1x3)A simply becomes (1x3)A - the (1x3) looks like a marker, but because it's within a data section of another marker, it is not treated any differently from the A that comes after it. It has a decompressed length of 6.
- X(8x2)(3x3)ABCY becomes X(3x3)ABC(3x3)ABCY (for a decompressed length of 18), because the decompressed data from the (8x2) marker (the (3x3)ABC) is skipped and not processed further.
- (4x5othertext(4x5)ABCD  I added this case in anticipation of what might be in the input file


In [ ]:
import re

pattern_marker = re.compile(r'\((\d+)x(\d+)\)')

def decompress_string(input_string):
    """
    Function to decompress a string
    """
    # convert to a list so we can work with it 
    input_list = list(input_string)
    # initialize new list for decompressed data
    new_list = []
    # idx will be a pointer to our location in the string, start at 0
    idx = 0
    # walk the string. note: we may increment idx within the loop, so not using enumerate
    while idx < len(input_list):
        # check to see if we might be at the start of a marker
        if input_list[idx] == '(':
            marker = '(';
            idx+=1
            # add to marker until we reach ')'
            while input_list[idx] != ')':
                marker += input_list[idx]
                idx+=1
            marker += ')'
            # regex to see if we have really found a marker or just a '('
            # note, there could be something really evil in the data like:
            #  (4x5othertext)
            # hopefully not really, really evil like:
            #  (4x5othertext(4x5), if so we will miss a marker, wait I think I can fix that
            match = pattern_marker.search(marker)
            if match:
                # fix for (4x5othertext(4x5) case
                if match.span()[0] != 0:
                    # add non-matching portion of marker back to new_list
                    new_list = new_list + list(f'{marker[0:match.span()[0]]}')
                char_count = int(match.group(1))
                multiplier = int(match.group(2))
                tmp_string = ''
                # get next char_count characters
                for _ in range(char_count):
                    idx+=1
                    tmp_string += input_list[idx]
                # add to newlist multiplier times
                new_list = new_list + list(tmp_string)*multiplier
                idx+=1
            else: # not a marker lets just stick it on the string
                new_list = new_list + list(f'({marker})')
        else:
            new_list.append(input_list[idx])
            idx += 1
    return ''.join(new_list)

In [ ]:
test_data = [
    'ADVENT',
    'A(1x5)BC',
    '(3x3)XYZ',
    'A(2x2)BCD(2x2)EFG',
    '(6x1)(1x3)A',
    'X(8x2)(3x3)ABCY',
    '(4x5othertext(4x5)ABCD'
]

In [100]:
for original_string in test_data:
    new_string = decompress_string(original_string)
    print(new_string,len(new_string))

ADVENT 6
ABBBBBC 7
XYZXYZXYZ 9
ABCBCDEFEFG 11
(1x3)A 6
X(3x3)ABC(3x3)ABCY 18
(4x5othertextABCDABCDABCDABCDABCD 33


In [32]:
pattern_marker = re.compile(r'\((\d+)x(\d+)\)')
marker = 'sometext(1x5)'
match = pattern_marker.search(marker)
print(match)
print(marker[0:match.span()[0]])

<re.Match object; span=(8, 13), match='(1x5)'>
sometext


In [ ]:
match.groups()
match.span()

In [ ]:
tmp_string = 'B'
multiplier=5
list(tmp_string)*multiplier

In [ ]:
from datetime import datetime


In [ ]:
datetime.today()

# Part 2
Apparently, the file actually uses version two of the format.

In version two, the only difference is that markers within decompressed data are decompressed. This, the documentation explains, provides much more substantial compression capabilities, allowing many-gigabyte files to be stored in only a few kilobyte
se:

(3x3)XYZ still becomes XYZXYZXYZ, as the decompressed section contains no mar
kers.
X(8x2)(3x3)ABCY becomes XABCABCABCABCABCABCY, because the decompressed data from the (8x2) marker is then further decompressed, thus triggering the (3x3) marker twice for a total of six ABC sequ
ences.
(27x12)(20x12)(13x14)(7x10)(1x12)A decompresses into a string of A repeated 241920
 times.
(25x3)(3x3)ABC(2x3)XY(5x2)PQRSTX(18x9)(3x2)TWO(5x7)SEVEN becomes 445 characte
rs long.
Unfortunately, the computer you brought probably doesn't have enough memory to actually decompress the file; you'll have to come up with another way to get its decompresseoved format?

In [ ]:
my_string='(27x12)(20x12)(13x14)(7x10)(1x12)A'

In [ ]:
new_string = decompress_string(new_string)
print(new_string)

In [ ]:
len(new_string)

In [135]:
# borrowed algorithm. rewritten from my notes
def decompress(input_string,v2=False):
    """
    recursive decompress function
	original algorithm credit: u/blockingthesky

    for this version I stripped out the code leaving my comments for guidance and rewrote the function

    Note, this does fail my super evil case: '(4x5othertext(4x5)ABCD', fortunately that doesn't seem
    to exist in the puzzle input.
    
    parameters:
      - input_string: string to decompress
      - v2: bool flag to use v2 recursion default False
    
    returns:
      - decompressed_length: integer length of decompressed data
    """
    # simple case input string does not have any markers, return its length
    if '(' not in input_string:
        return len(input_string)
    # initialize decompressed_length
    decompressed_length = 0
    match = pattern_marker.search(input_string)
    while match:
        #<re.Match object; span=(1, 6), match='(1x5)'>
        # add characters up to next marker
        decompressed_length += int(match.span()[0])
        # extract marker
        span=int(match.group(1))
        multiplier=int(match.group(2))
        next_span = input_string[int(match.span()[1]):match.span()[1]+span]
        # for part 2, lets get recursive
        if v2:
            # decompress next span characters multiplied by multiplier
            decompressed_length += decompress(next_span,v2)*multiplier
        else:
            # add length of span characters multiplied by multiplier
            decompressed_length += len(next_span)*multiplier
        # remove processed string portion
        input_string = input_string[match.span()[1]+span:]
        # reprocess match (leave commented for now to avoid infinite loop)
        match = pattern_marker.search(input_string)
    # add length of remaining string
    decompressed_length += len(input_string)
    
    return decompressed_length

In [36]:
print(decompress('(27x12)(20x12)(13x14)(7x10)(1x12)A'))

0


In [24]:
test_data

['ADVENT',
 'A(1x5)BC',
 '(3x3)XYZ',
 'A(2x2)BCD(2x2)EFG',
 '(6x1)(1x3)A',
 'X(8x2)(3x3)ABCY',
 '(4x5othertext(4x5)ABCD']

In [42]:
for original_string in test_data:
    decompressed_size = decompress(original_string)
    print(original_string,decompressed_size)

ADVENT 6
<re.Match object; span=(1, 6), match='(1x5)'>
A(1x5)BC 0
<re.Match object; span=(0, 5), match='(3x3)'>
(3x3)XYZ 0
<re.Match object; span=(1, 6), match='(2x2)'>
A(2x2)BCD(2x2)EFG 0
<re.Match object; span=(0, 5), match='(6x1)'>
(6x1)(1x3)A 0
<re.Match object; span=(1, 6), match='(8x2)'>
X(8x2)(3x3)ABCY 0
<re.Match object; span=(13, 18), match='(4x5)'>
(4x5othertext(4x5)ABCD 0


In [86]:
input_string='A(1x5)BC'
match = pattern_marker.search(input_string)
print(f"adding before match: {int(match.span()[0])}")



adding before match: 1


In [56]:
match.group(1)

'1'

In [80]:
input_string[6:8]

'BC'

In [106]:
for input_string in test_data:
    print(input_string)
    print(decompress(input_string))


ADVENT
6
A(1x5)BC
next_span = B
input_string = C
7
(3x3)XYZ
next_span = XYZ
input_string = 
9
A(2x2)BCD(2x2)EFG
next_span = BC
input_string = D(2x2)EFG
next_span = EF
input_string = G
11
(6x1)(1x3)A
next_span = (1x3)A
input_string = 
6
X(8x2)(3x3)ABCY
next_span = (3x3)ABC
input_string = Y
18
(4x5othertext(4x5)ABCD
next_span = ABCD
input_string = 
33


In [127]:
test_data2 = [
    '(3x3)XYZ',
    'X(8x2)(3x3)ABCY',
    '(27x12)(20x12)(13x14)(7x10)(1x12)A',
    '(25x3)(3x3)ABC(2x3)XY(5x2)PQRSTX(18x9)(3x2)TWO(5x7)SEVEN'
]

In [133]:
for input_string in test_data2:
    print(input_string)
    print(decompress(input_string,True))

(3x3)XYZ
9
X(8x2)(3x3)ABCY
20
(27x12)(20x12)(13x14)(7x10)(1x12)A
241920
(25x3)(3x3)ABC(2x3)XY(5x2)PQRSTX(18x9)(3x2)TWO(5x7)SEVEN
445
